In [27]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [28]:
pages = [
    "https://www.amazon.in/OnePlus-Nord-Black-128GB-Storage/product-reviews/B09WQY65HN/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews",
    "https://www.amazon.in/OnePlus-Nord-Black-128GB-Storage/product-reviews/B09WQY65HN/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2",
    "https://www.amazon.in/OnePlus-Nord-Black-128GB-Storage/product-reviews/B09WQY65HN/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3"
]

headers = {
    'authority': 'www.amazon.in',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'session-id=259-3113978-6678618; i18n-prefs=INR; ubid-acbin=260-8554202-6973909; lc-acbin=en_IN; csm-hit=tb:BS866TA0AKH6X86N924E+sa-7XYTQAXQHJP5ADH88228-DY27HYE0CK5V9FW24GBD|1656009294944&t:1656009294945&adb:adblk_yes; session-token=Z1j175VoYxPr2Un/9ciL3Q6lKw+QtLYYIwSQ+GLxjT06952u8vOZromD4WcFE0bs+yrUyLPy8HmIn7mTjUt8qsx3n0meC7yWKFqqwDEm5iecYedklsrNwmDrQOiaMH9lpacbdB8kgUk5IbZdg1VyhrdnY4OZrk6r350ARDEXJExuu2GZr0sV4fpbwUes/V9fDrfASeMQhVEEzmEAAHWN2g==; session-id-time=2082758401l',
    'device-memory': '8',
    'downlink': '10',
    'dpr': '0.8',
    'ect': '4g',
    'referer': 'https://www.amazon.in/OnePlus-Nord-Black-128GB-Storage/dp/B09WQY65HN/ref=sr_1_4?crid=1D99WHM86WX80&keywords=oneplus&qid=1656009113&sprefix=onep%2Caps%2C315&sr=8-4&th=1',
    'rtt': '0',
    'sec-ch-device-memory': '8',
    'sec-ch-dpr': '0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-ch-viewport-width': '2400',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'service-worker-navigation-preload': 'true',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
    'viewport-width': '2400',
}

In [29]:
resp = requests.get(pages[0], headers=headers)
resp

<Response [200]>

In [30]:
soup = BeautifulSoup(resp.text, "html.parser") # html.parser, lxml

In [31]:
# 10 divs
reviews = soup.find_all("div", {"class": "a-section review aok-relative"})

In [32]:
def get_user_name(soup_obj: BeautifulSoup) -> str:
    try:
        user_name = soup_obj.find('span', {"class": "a-profile-name"}).get_text()
        return user_name
    except Exception as e:
        return 'not_found'
        print(e)

In [33]:
def get_review_stars(soup_obj: BeautifulSoup) -> str:
    try:
        review_str = soup_obj.find('a', {"class": "a-link-normal"}).get_text()
        return review_str
    except Exception as e:
        return 'not_found'
        print(e)

In [34]:
def get_review_title(soup_obj: BeautifulSoup) -> str:
    try:
        review_title = soup_obj.find('a', 
            {"class": "a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold"}
        ).get_text().strip()
        return review_title
    except Exception as e:
        return 'not_found'
        print(e)

In [35]:
def get_review_body(soup_obj: BeautifulSoup) -> str:
    try:
        review_body = soup_obj.find('span', 
            {"class": "a-size-base review-text review-text-content"}
        ).get_text().strip()
        return review_body
    except Exception as e:
        return 'not_found'
        print(e)

In [36]:
def get_review_date(soup_obj: BeautifulSoup) -> str:
    try:
        review_date = soup_obj.find('span', 
            {"class": "a-size-base a-color-secondary review-date"}
        ).get_text().strip()
        return review_date
    except Exception as e:
        return 'not_found'
        print(e)

In [37]:
def get_review_components(soup_obj: BeautifulSoup) -> dict:
    return {
        'User': get_user_name(soup_obj) , 
        'Rating': get_review_stars(soup_obj), 
        'Title': get_review_title(soup_obj),
        'Review': get_review_body(soup_obj),
        'Date': get_review_date(soup_obj)
    }

In [38]:
records = [get_review_components(rev) for rev in reviews]
df = pd.DataFrame.from_records(records)
#df.to_csv('amazon_reviews.csv')
df

,User,Rating,Title,Review,Date
0,KAMAL VIRANI,2.0 out of 5 stars,Worst mobile not satisfied,Worst phone ever seen. Not upto the mark. Retu...,Reviewed in India on 4 May 2022
1,Verified purchase,4.0 out of 5 stars,"Good in budget, not best.",The fingerprint sensor does not response somet...,Reviewed in India on 4 May 2022
2,rudri srihari,5.0 out of 5 stars,Good,The media could not be loaded.\n ...,Reviewed in India on 4 May 2022
3,Reshu Rani,4.0 out of 5 stars,Good phone with all advanced features in best ...,The media could not be loaded.\n ...,Reviewed in India on 8 May 2022
4,Ranjeet Kumar,4.0 out of 5 stars,My First Sasta Phone 😊,"It seems good for now, let us explore all feat...",Reviewed in India on 4 May 2022
5,NASH,1.0 out of 5 stars,Crap product.,Oxygen OS has watered down and OnePlus is no l...,Reviewed in India on 5 May 2022
6,Ankit Koshti,4.0 out of 5 stars,A great phone for family entertainment,One plus phones have always amazed me by the t...,Reviewed in India on 5 May 2022
7,r,1.0 out of 5 stars,Don't go for it ...,I am unable to export or import contacts with ...,Reviewed in India on 5 May 2022
8,Debjyoti Mallick,4.0 out of 5 stars,Good battery Backup.Good Camera .Everything is...,The media could not be loaded.\n ...,Reviewed in India on 4 May 2022
9,sri,5.0 out of 5 stars,Strong value phone,Great battery. Only 40% down since a day (not ...,Reviewed in India on 6 May 2022


In [40]:
ALL_REVIEWS = []

for u in pages:
    print(u)
    resp = requests.get(u, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    reviews = soup.find_all("div", {"class": "a-section review aok-relative"})
    records = [get_review_components(rev) for rev in reviews]
    ALL_REVIEWS.extend(records) 
    
    
out = pd.DataFrame.from_records(ALL_REVIEWS)
out

https://www.amazon.in/OnePlus-Nord-Black-128GB-Storage/product-reviews/B09WQY65HN/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.in/OnePlus-Nord-Black-128GB-Storage/product-reviews/B09WQY65HN/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.in/OnePlus-Nord-Black-128GB-Storage/product-reviews/B09WQY65HN/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3


,User,Rating,Title,Review,Date
0,KAMAL VIRANI,2.0 out of 5 stars,Worst mobile not satisfied,Worst phone ever seen. Not upto the mark. Retu...,Reviewed in India on 4 May 2022
1,Verified purchase,4.0 out of 5 stars,"Good in budget, not best.",The fingerprint sensor does not response somet...,Reviewed in India on 4 May 2022
2,rudri srihari,5.0 out of 5 stars,Good,The media could not be loaded.\n ...,Reviewed in India on 4 May 2022
3,Reshu Rani,4.0 out of 5 stars,Good phone with all advanced features in best ...,The media could not be loaded.\n ...,Reviewed in India on 8 May 2022
4,Ranjeet Kumar,4.0 out of 5 stars,My First Sasta Phone 😊,"It seems good for now, let us explore all feat...",Reviewed in India on 4 May 2022
5,NASH,1.0 out of 5 stars,Crap product.,Oxygen OS has watered down and OnePlus is no l...,Reviewed in India on 5 May 2022
6,Ankit Koshti,4.0 out of 5 stars,A great phone for family entertainment,One plus phones have always amazed me by the t...,Reviewed in India on 5 May 2022
7,r,1.0 out of 5 stars,Don't go for it ...,I am unable to export or import contacts with ...,Reviewed in India on 5 May 2022
8,Debjyoti Mallick,4.0 out of 5 stars,Good battery Backup.Good Camera .Everything is...,The media could not be loaded.\n ...,Reviewed in India on 4 May 2022
9,sri,5.0 out of 5 stars,Strong value phone,Great battery. Only 40% down since a day (not ...,Reviewed in India on 6 May 2022


## Looping through multiple pages
One of the easiest methods to scrape multiple pages is to modify the base URL to accept a page variable that increments as needed.

Try for yourself! See how the URL changes as you go through multiple pages.

For Amazon product reviews, the only thing that seems to change is the number indicating which page it is.

#### Okay, now let’s put this to work in a function:

In [4]:
def get_soup(url):
    #r = requests.get('http://localhost:8050/render.html', 
    # Run this instead if you haven't setup Splash & Docker
    r = requests.get(url, headers=headers,
    params={'url': url, 'wait': 2})
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

In [5]:
# Initialize list to store reviews data later on
reviewlist = []

# Function 2: look for web-tags in our soup, then append our data to reviewList
def get_reviews(soup):
    reviews = soup.find_all('div', {'data-hook': 'review'})
    try:
        for item in reviews:
            review = {
            'product': soup.title.text.replace('Amazon.in:Customer reviews: ', '').strip(),    
            'date': item.find('span', {'data-hook': 'review-date'}).text.replace('Reviewed in India on', '').strip(),
            'title': item.find('a', {'data-hook': 'review-title'}).text.strip(),
            'rating':  float(item.find('i', {'data-hook': 'review-star-rating'}).text.replace('out of 5 stars', '').strip()),
            'body': item.find('span', {'data-hook': 'review-body'}).text.strip(),
            }
            reviewlist.append(review)
    except:
        pass

In [12]:
# Initialize list to store reviews data later on
reviewlist = []

# Function 2: look for web-tags in our soup, then append our data to reviewList
def get_reviews(soup):
    reviews = soup.find_all('div', {'data-hook': 'review'})
    try:
        for item in reviews:
            review = {
            'Rating':  float(item.find('i', {'data-hook': 'review-star-rating'}).text.replace('out of 5 stars', '').strip()),
            'Title': item.find('a', {'data-hook': 'review-title'}).text.strip(),
            'Review': item.find('span', {'data-hook': 'review-body'}).text.strip(),
            'Review_Date': item.find('span', {'data-hook': 'review-date'}).text.replace('Reviewed in India on', '').strip(),
            }
            reviewlist.append(review)
    except:
        pass

In [13]:
# loop through 1:x many pages, or until the css selector found only on the last page is found (when the next page button is greyed)
for x in range(1,5):
    soup = get_soup(f'https://www.amazon.in/OnePlus-Nord-Black-128GB-Storage/product-reviews/B09WQY65HN/\
ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber={x}')
    print(f'Getting page: {x}')
    get_reviews(soup)
    print(len(reviewlist))

Getting page: 1
10
Getting page: 2
20
Getting page: 3
30
Getting page: 4
40


In [14]:
# Save results to a dataframe, then export as CSV
df = pd.DataFrame(reviewlist)
#df.to_csv(r'sony-headphones.csv', index=False)
df

,Rating,Title,Review,Review_Date
0,2.0,Worst mobile not satisfied,Worst phone ever seen. Not upto the mark. Retu...,4 May 2022
1,4.0,"Good in budget, not best.",The fingerprint sensor does not response somet...,4 May 2022
2,5.0,Good,The media could not be loaded.\n ...,4 May 2022
3,4.0,Good phone with all advanced features in best ...,The media could not be loaded.\n ...,8 May 2022
4,4.0,My First Sasta Phone 😊,"It seems good for now, let us explore all feat...",4 May 2022
5,1.0,Crap product.,Oxygen OS has watered down and OnePlus is no l...,5 May 2022
6,4.0,A great phone for family entertainment,One plus phones have always amazed me by the t...,5 May 2022
7,1.0,Don't go for it ...,I am unable to export or import contacts with ...,5 May 2022
8,4.0,Good battery Backup.Good Camera .Everything is...,The media could not be loaded.\n ...,4 May 2022
9,5.0,Strong value phone,Great battery. Only 40% down since a day (not ...,6 May 2022


#### We can even add in a stop condition. For this one, we can tell Python to look for a greyed out “Next Page” button. To identify this element, use the element inspector.

Add this to the bottom of the function above.

In [ ]:
# loop through 1:x many pages, or until the css selector found only on the last page is found (when the next page button is greyed)
for x in range(1,20):
    soup = get_soup(f'https://www.amazon.ca/Sony-WF-1000XM3-Industry-Canceling-Wireless/product-reviews/B07T81554H/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber={x}')
    print(f'Getting page: {x}')
    get_reviews(soup)
    print(len(reviewlist))
    if not soup.find('li', {'class': 'a-disabled a-last'}):
        pass
    else:
        break